Create a dataset for @HOME2024 using the mask provided in the YCB dataset and SAM.

In [ ]:
import os
import random
import json
import numpy as np
import cv2
from PIL import Image, ImageDraw, ImageEnhance, ImageFilter, ImageFont
from pycocotools import mask
import json
import yaml
import csv
import torch
import matplotlib.pyplot as plt
from pathlib import Path
import ultralytics
import time
import imutils
import argparse

In [ ]:

def remove_background(image_path, mask_path, output_path):
    # Carga la imagen y la máscara
    image = cv2.imread(image_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # Invierte la máscara
    inverted_mask = cv2.bitwise_not(mask)

    # threshold input image using otsu thresholding as mask and refine with morphology
    ret, pngmask = cv2.threshold(inverted_mask, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU) 
    kernel = np.ones((9,9), np.uint8)
    pngmask = cv2.morphologyEx(pngmask, cv2.MORPH_CLOSE, kernel)
    pngmask = cv2.morphologyEx(pngmask, cv2.MORPH_OPEN, kernel)

    # Copia los píxeles de la imagen original a la imagen transparente solo donde la máscara es diferente de cero
    result = image.copy()
    result = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
    result[:, :, 3] = pngmask                           

    # Guarda el resultado en formato PNG para preservar la transparencia
    cv2.imwrite(output_path, result, [cv2.IMWRITE_PNG_COMPRESSION, 9])

# Rutas de los archivos de entrada y salida
filepath = '/Users/josebenvenuto/Desktop/tempo/'
mask_path = '/Users/josebenvenuto/Desktop/mask/'
output_path = '/Users/josebenvenuto/Desktop/tempo/salida/.png'

imgPaths = os.listdir(filepath)

for img in imgPaths:
    image_path = filepath + img
    mask_path = mask_path + img
    output_path = filepath + img
    remove_background(image_path, mask_path, output_path)


# Remueve el fondo y guarda la imagen resultante
remove_background(image_path, mask_path, output_path)


In [ ]:
from PIL import Image
import os

datasetpath = "/home/jabv/Desktop/prueba/res/"
resultspath = "/home/jabv/Desktop/prueba/DS_res/"

#create a result folder if it doesn't exist
if not os.path.exists(resultspath):
    os.makedirs(resultspath)
    os.makedirs(resultspath+"gatos/")
    os.makedirs(resultspath+"jarra/")
    os.makedirs(resultspath+"lata/")
    os.makedirs(resultspath+"manzana/")
    os.makedirs(resultspath+"platano/")
    os.makedirs(resultspath+"silla/")


fg_folders = [
    ("gatos/"),
    ("jarra/"),
    ("lata/"),
    ("manzana/"),
    ("platano/"),
    ("silla/")
]

for folder in fg_folders:
    for filename in os.listdir(f"{datasetpath}{folder}"):
        try:
            print(f"{filename} started")
            myImage = Image.open(datasetpath+folder+filename)
            black = Image.new('RGBA', myImage.size)
            myImage = Image.composite(myImage, black, myImage)
            #print("aqui")
            myCroppedImage = myImage.crop(myImage.getbbox())
            myCroppedImage.save(f"{resultspath}{folder}{filename}")
            print(f"{filename} done")
        except:
            print(f"{filename} failed")
            continue
print("all done")